In [5]:
# 1. 导入必要的库

import numpy as np
import pandas as pd
import scipy.stats as stats
import joblib
import pickle
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import sys
from typing import Dict, Any

# 导入自定义模块
import MyNet
from MyNet import NetList, DNN, LogReg

In [15]:
def load_and_analyze_model(file_path):
    try:
        data = joblib.load(file_path)
        print("File loaded successfully")
        
        print("\nData Structure Analysis:")
        for key, value in data.items():
            print(f"\n{key}:")
            if isinstance(value, NetList):
                print(f"  Type: NetList containing {len(value)} models")
                print(f"  First model type: {type(value.models[0])}")
            elif isinstance(value, list):
                print(f"  Type: list of length {len(value)}")
                if value:
                    print(f"  First item type: {type(value[0])}")
            elif isinstance(value, np.ndarray):
                print(f"  Type: numpy array of shape {value.shape}")
            elif isinstance(value, (int, float)):
                print(f"  Type: {type(value)}, Value: {value}")
            else:
                print(f"  Type: {type(value)}")
        
        if 'train_losses' in data and 'main_losses' in data:
            plt.figure(figsize=(12, 6))
            plt.plot(data['train_losses'], label='Training Loss')
            plt.plot(np.arange(0, len(data['train_losses']), len(data['train_losses'])//len(data['main_losses'])), 
                     data['main_losses'], label='Validation Loss')
            plt.xlabel('Training Steps')
            plt.ylabel('Loss')
            plt.title('Training and Validation Losses')
            plt.legend()
            plt.grid(True)
            plt.savefig('loss_plot.png')
            plt.close()
            print("\nLoss plot saved as 'loss_plot.png'")
        
        return data
    except Exception as e:
        print(f"Error during loading or analysis: {e}")
        return None

def generate_report(res: Dict[str, Any]) -> str:
    report = "实验数据分析报告\n" + "================\n\n"
    # 1. 训练信息
    report += "1. 训练信息\n"
    report += "-------------\n"

    steps = len(res['info'])
    report += f"总训练步数: {steps}\n"
    batch_size = len(res['info'][0]['idx'])
    report += f"batch size: {batch_size}\n"
    epochs =len(res['main_losses']) - 1
    report += f"epoch number: {epochs}\n"
    steps_per_epoch = steps / epochs
    report += f"每轮步数: {steps_per_epoch:.0f}\n\n"
    
        

    # 2. 模型信息
    report += "2. 模型信息\n"
    report += "-------------\n"
    report += f"模型类型: {type(res['models'])}\n"
    report += f"保存的模型数量: {len(res['models'])}\n\n"

    # 3. 损失信息
    report += "3. 损失信息\n"
    report += "-------------\n"
    report += "主模型损失:\n"
    report += f"  类型: {type(res['main_losses'])}\n"
    report += f"  记录次数: {len(res['main_losses'])}\n"
    report += f"  最终损失: {res['main_losses'][-1]:.4f}\n\n"
    report += "训练损失:\n"
    report += f"  类型: {type(res['train_losses'])}\n"
    report += f"  形状: {res['train_losses'].shape}\n"
    report += f"  最终损失: {res['train_losses'][-1]:.4f}\n\n"

    # 4. 反事实分析
    report += "4. 反事实分析\n"
    report += "-------------\n"
    report += f"分析的样本数: {len(res['counterfactual'])}\n"
    report += f"每个样本的模型状态数: {len(res['counterfactual'][0])}\n\n"

    # 5. 其他信息
    report += "5. 其他信息\n"
    report += "-------------\n"
    report += f"Alpha 值: {res['alpha']}\n"

    return report

def generate_combined_loss_plot(res: Dict[str, Any], save_path: str):
    fig, ax = plt.subplots(figsize=(12, 8))

    # Calculate epochs and steps
    # epochs = len(res['info']) / (200 / len(res['info'][0]['idx']))
    steps = len(res['info'])
    # batch_size = len(res['info'][0]['idx'])
    epochs =len(res['main_losses']) - 1
    steps_per_epoch = steps / epochs

    # Training loss plot
    x_train = np.arange(len(res['train_losses'])) / steps_per_epoch
    ax.plot(x_train, res['train_losses'], label='Training Loss', alpha=0.7)

    # Main model loss plot
    x_main = np.arange(len(res['main_losses']))
    ax.plot(x_main, res['main_losses'], label='Main Model Loss', alpha=0.7)

    # Set labels and title
    ax.set_title('Training and Main Model Losses vs. Epochs', fontsize=16)
    ax.set_xlabel('Epochs', fontsize=12)
    ax.set_ylabel('Loss', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend(fontsize=10)

    # Add text with additional information
    info_text = f"Total Epochs: {epochs:.1f}\n" \
                f"Steps per Epoch: {steps_per_epoch:.0f}\n" \
                f"Batch Size: {len(res['info'][0]['idx'])}\n" \
                f"Alpha: {res['alpha']}"
    ax.text(0.95, 0.95, info_text, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Loss plot saved to: {save_path}")

In [17]:
sys.modules['MyNet'] = sys.modules[__name__]

# 定义文件路径
# path = '/home/zihan/codes/sgd-influence/experiment/Sec71/20news_dnn/sgd002.dat'
path = '/home/zihan/codes/sgd-influence/experiment/Sec71/20news_logreg/sgd001.dat'

# 尝试加载数据
try:
    res = joblib.load(path, mmap_mode='r')
    print("数据加载成功")
    print("数据键:", res.keys())
except Exception as e:
    print(f"Joblib 加载失败: {e}")
    try:
        with open(path, 'rb') as f:
            res = pickle.load(f)
        print("Pickle 加载成功")
        print("数据键:", res.keys())
    except Exception as e:
        print(f"Pickle 加载失败: {e}")
        res = None

if res is None:
    print("无法加载数据，请检查文件路径和格式")


/home/zihan/codes/sgd-influence/.venv/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


数据加载成功
数据键: dict_keys(['models', 'info', 'counterfactual', 'alpha', 'main_losses', 'train_losses'])


In [18]:

# 4. 数据分析

if res is not None:
    # 生成报告
    report = generate_report(res)
    print(report)

    # 生成损失图
    generate_combined_loss_plot(res, path.replace('.dat', '.png'))

# 5. 额外分析（可选）

# 这里可以添加任何额外的分析代码
# 例如，探索模型结构，分析特定参数等

# 示例：打印模型结构（如果适用）
if 'models' in res and isinstance(res['models'], NetList):
    print("\n模型结构:")
    print(res['models'].models[0])  # 假设第一个模型代表整体结构

# 6. 结论和下一步

print("\n分析结论:")
print("1. 模型训练过程中的损失变化已可视化，可以观察训练和验证损失的趋势。")
print("2. 生成的报告提供了关于模型训练的关键信息，包括训练轮数、批次大小等。")
print("3. 反事实分析部分显示了每个样本的模型状态数，这可能对理解模型的鲁棒性很有帮助。")

print("\n下一步计划:")
print("1. 深入分析反事实模型，比较不同样本对模型训练的影响。")
print("2. 考虑添加更多的可视化，例如参数分布图或学习率变化图。")
print("3. 如果可能，进行交叉验证或在不同数据集上测试模型的泛化能力。")


实验数据分析报告

1. 训练信息
-------------
总训练步数: 480
batch size: 5
epoch number: 12
每轮步数: 40

2. 模型信息
-------------
模型类型: <class 'MyNet.NetList'>
保存的模型数量: 481

3. 损失信息
-------------
主模型损失:
  类型: <class 'list'>
  记录次数: 13
  最终损失: 0.6328

训练损失:
  类型: <class 'numpy.memmap'>
  形状: (481,)
  最终损失: 0.4563

4. 反事实分析
-------------
分析的样本数: 200
每个样本的模型状态数: 481

5. 其他信息
-------------
Alpha 值: 6.45774832507442

Loss plot saved to: /home/zihan/codes/sgd-influence/experiment/Sec71/20news_logreg/sgd001.png

模型结构:
LogReg(
  (fc): Linear(in_features=4752, out_features=1, bias=True)
)

分析结论:
1. 模型训练过程中的损失变化已可视化，可以观察训练和验证损失的趋势。
2. 生成的报告提供了关于模型训练的关键信息，包括训练轮数、批次大小等。
3. 反事实分析部分显示了每个样本的模型状态数，这可能对理解模型的鲁棒性很有帮助。

下一步计划:
1. 深入分析反事实模型，比较不同样本对模型训练的影响。
2. 考虑添加更多的可视化，例如参数分布图或学习率变化图。
3. 如果可能，进行交叉验证或在不同数据集上测试模型的泛化能力。


In [8]:
res

{'models': NetList(
   (models): ModuleList(
     (0-600): 601 x LogReg(
       (fc): Linear(in_features=4752, out_features=1, bias=True)
     )
   )
 ),
 'info': [{'idx': memmap([993, 859, 298, 553, 672, 971,  27, 231, 306, 706, 496, 558, 784,
           239, 578,  55, 906, 175,  14,  77]),
   'lr': 0.01},
  {'idx': memmap([ 31, 481, 310, 311, 883, 788,  45, 103, 760,   1, 823, 710, 614,
           790, 408, 736, 957, 366, 918, 267]),
   'lr': 0.007071067811865476},
  {'idx': memmap([230, 996, 635, 698, 251, 783, 819, 141, 316, 587, 331, 295, 262,
           432, 862, 582, 272, 270, 987, 319]),
   'lr': 0.005773502691896259},
  {'idx': memmap([569, 643, 142, 202, 413, 196, 264, 531, 252, 576, 738, 299, 740,
           247, 926, 412, 389, 796, 601, 654]),
   'lr': 0.005000000000000001},
  {'idx': memmap([261, 456, 386, 982, 909, 693, 236, 501, 497, 874, 452, 494, 923,
           279, 638, 485, 568, 108, 367, 644]),
   'lr': 0.00447213595499958},
  {'idx': memmap([785, 873,  65, 902, 31